# Stage 4: Matching
## Import required packages


In [1]:
# import py_entitymatching package
import py_entitymatching as em
import pandas as pd
import re
import nltk

/home/karan/anaconda2/envs/npy/lib/python2.7/site-packages/scikit_learn-0.18.1-py2.7-linux-x86_64.egg/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


## Read the input tables

In [2]:
# specify filepaths for tables A and B. 
path_A = 'spoj/json/spoj_blocking.csv' # patoh_A is the file path where table_A.csv is stored
path_B = 'codechef/data/codechef_blocking.csv' # path_B is the file path where table_B.csv is stored

In [3]:
# read table A; table A has 'ID' as the key attribute
A = em.read_csv_metadata(path_A)

# read table B; table B has 'ID' as the key attribute
B = em.read_csv_metadata(path_B)
A.rename(columns={"Unnamed: 0":"ID"}, inplace=True)
B.rename(columns={"Unnamed: 0":"ID"}, inplace=True)

No handlers could be found for logger "py_entitymatching.io.parsers"


In [4]:
A["words"] = A["description"].fillna("") + A["input"].fillna("") + A["output"].fillna("")
A = A.drop(["description", "input", "output"], axis=1)
B["words"] = B["description"].fillna("") + B["input"].fillna("") + B["output"].fillna("")
B = B.drop(["description", "input", "output"], axis=1)

In [5]:
# Step 1
def std_words(df):
    letters_only = re.sub("[^a-zA-Z]", " ", df["words"]) # letters only, drop numbers & symbols
    words = letters_only.lower().split() # lower case, split by word                  
    stops = set((nltk.corpus.stopwords.words("english") + 
                ['a','b','c','d','e','f','g','h','i','j','k','l','m','n',
                'o','p','q','r','s','t','u','v','w','x','y','z'])) # general stopwords & letters  
    meaningful_words = [w for w in words if not w in stops] # remove stopwords
    return( " ".join( meaningful_words)) # return re-joined string

# Step 2
A["words"] = A.apply(std_words, axis=1)
B["words"] = B.apply(std_words, axis=1)
em.set_key(A, "ID")
em.set_key(B, "ID")

True

## Read in candidate set

In [6]:
# read C
C = em.read_csv_metadata('candidate_set.csv', ltable=A, rtable=B)

KeyError: 'Input key ( _id ) not in the DataFrame'

In [ ]:
table_A = pd.read_csv("spoj/json/problems.csv")
table_B = pd.read_csv("codechef/data/codechef_problems.csv")

In [ ]:
C["ltable_url"] = table_A.loc[C.ltable_ID.values,"url"].values
C["rtable_url"] = table_B.loc[C.rtable_ID.values,"link"].values

In [ ]:
C.rename(inplace=True,
         columns={"ltable_title":"ltable_problem_code",
                  "rtable_title":"rtable_problem_code"})

In [ ]:
C["ltable_problem_title"] = table_A.loc[C.ltable_ID.values,"title"].apply(lambda x:x.split(" - ")[1]).values
C["rtable_problem_title"] = table_B.loc[C.rtable_ID.values,"title"].values

In [ ]:
D = C.sample(400, random_state=44)

In [ ]:
D.head()

In [7]:
D.loc[:,["ltable_url","ltable_problem_code","rtable_url","rtable_problem_code"]].to_csv('labelled_set.csv', index=False)

NameError: name 'D' is not defined